# Ekstraksi Informasi dari Berita Kriminal Indonesia menggunakan Named Entity Recognition (NER).

In [1]:
import pandas as pd
import re
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [8]:
df = pd.read_csv('../data/processed/cleaned_all_data.csv')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764
2,https://www.detik.com/jatim/hukum-dan-kriminal...,"Kriminalitas Surabaya Marak, Pengamat Sebut Pe...",2024-09-05 02:01:00,Sejumlah kejadian kriminalitas kian marak terj...,72,1224
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398


In [10]:
df1 = pd.read_csv('../data/processed/preprocessed.csv')
df1.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed,judul_stop_removed,isi_stop_removed,judul_pos_tags,isi_pos_tags
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"['4', 'Anak', 'di', 'Bawah', 'Umur', 'Tersangk...","['Pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'tersangk...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'sangka',...","['laku', 'bunuh', 'dan', 'perkosa', 'aa', '14'...","['4', 'anak', 'umur', 'sangka', 'bunuh', 'pale...","['laku', 'bunuh', 'perkosa', 'aa', '14', 'oran...","[('4', 'NUM'), ('anak', 'NOUN'), ('umur', 'NOU...","[('laku', 'NOUN'), ('bunuh', 'VERB'), ('perkos..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"['26', 'Tersangka', 'Dibekuk', 'Selama', '3', ...","['Dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'tersangka', 'dibekuk', 'selama', '3', ...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'sangka', 'bekuk', 'lama', '3', 'bulan'...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'sangka', 'bekuk', '3', 'kota', 'probol...","['3', 'polres', 'probolinggo', 'kota', 'bekuk'...","[('26', 'NUM'), ('sangka', 'NOUN'), ('bekuk', ...","[('3', 'NUM'), ('polres', 'NOUN'), ('proboling..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"['Pekerja', 'Kafe', 'Disiram', 'Air', 'Keras',...","['Seorang', 'pekerja', 'kafe', 'di', 'Cengkare...","['pekerja', 'kafe', 'disiram', 'air', 'keras',...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","['kerja', 'kafe', 'siram', 'air', 'keras', 'hi...","['orang', 'kerja', 'kafe', 'di', 'cengkareng',...","['kerja', 'kafe', 'siram', 'air', 'keras', 'wa...","['orang', 'kerja', 'kafe', 'cengkareng', 'mas'...","[('kerja', 'NOUN'), ('kafe', 'NOUN'), ('siram'...","[('orang', 'NOUN'), ('kerja', 'NOUN'), ('kafe'..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"['Hilang', 'Nyawa', 'Pria', 'di', 'Simalungun'...","['Hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebut', 'mikrofon', 'untu...","['hilang', 'nyawa', 'pria', 'simalungun', 'geg...","['gegara', 'rebut', 'mikrofon', 'nyanyi', 'war...","[('hilang', 'VERB'), ('nyawa', 'NOUN'), ('pria...","[('gegara', 'NOUN'), ('rebut', 'VERB'), ('mikr..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"['Utang', 'Rp', '3', 'Juta', 'Bikin', 'Pegawai...","['Hidup', 'pria', 'bernama', 'Monika', 'Hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'bernama', 'monika', 'hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'nama', 

In [15]:
df1['id'] = ['D{:04d}'.format(i+1) for i in range(len(df1))]

In [27]:
import ast
result = pd.DataFrame(columns=['id', 'token', 'entity'])

for i in range(len(df1)):
    id = df1['id'][i]
    print(id)
    
    tokens_string = df1['isi_tokens'][i]
    
    try:
        tokens_list = ast.literal_eval(tokens_string)
        
        for token in tokens_list:
            if isinstance(token, str):
                token = re.sub(r'\s+', ' ', token).strip()
                print(token)
                result = pd.concat([result, pd.DataFrame({'id': [id], 'token': [token], 'entity': [None]})], ignore_index=True)
    
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing tokens for {id}: {e}")

D0001
Pelaku
pembunuhan
dan
pemerkosaan
AA
14
seorang
remaja
putri
di
Kuburan
Cina
Palembang
sudah
ditangkap
dan
ditetapkan
tersangka
Keempat
tersangka
merupakan
anak
di
bawah
umur
sama
seperti
korbannya
Pengamat
Hukum
dari
Universitas
Taman
Siswa
Palembang
Dr
Azwar
Agus
menyampaikan
turut
prihatin
dengan
kejadian
pembunuhan
remaja
putri
yang
dilakukan
oleh
remaja
lakilaki
yang
usianya
sebaya
dengan
korban
Kejadian
ini
sangat
memprihatinkan
Dilihat
dari
aspek
hukum
kejahatan
ini
tidak
bisa
terbendung
seiring
dengan
perkembangan
ilmu
pengetahuan
dan
teknologi
seperti
peran
media
sosial
di
mana
salah
satu
tersangka
bisa
mengakses
film
biru
porno
ujarnya
Kamis
592024
SCROLL
TO
CONTINUE
WITH
CONTENT
Menurutnya
kontenkonten
negatif
seperti
seksual
tidak
bisa
dihindari
dan
ini
menjadi
pemicu
kejahatan
karena
terbukti
anak
tersebut
bisa
mengakses
kontenkonten
pornografi
dan
memicu
tindak
pidana
Sebenarnya
itu
konten
film
porno
tidak
hanya
bisa
diakses
anakanak
saja
tetapi
juga
bisa
diakses
or

In [28]:
result.to_csv('../data/processed/pseudolabelling-nopostag.csv', index=False)

In [29]:
pd.read_csv('../data/processed/pseudolabelling-nopostag.csv').head()

,id,token,entity
0,D0001,Pelaku,NaN
1,D0001,pembunuhan,NaN
2,D0001,dan,NaN
3,D0001,pemerkosaan,NaN
4,D0001,AA,NaN
